## Part 1: Preprocessing

In [89]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [90]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [91]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [92]:
# Create a list of at least 10 column names to use as X data
x_columns = [
    'Education',
    'Age',
    'DistanceFromHome',
    'JobSatisfaction',
    'OverTime',
    'StockOptionLevel',
    'WorkLifeBalance',
    'YearsAtCompany',
    'YearsSinceLastPromotion',
    'NumCompaniesWorked'
]

# Create X_df using your selected columns
X_df = attrition_df[x_columns]

# Show the data types for X_df
X_df.head()

,Education,Age,DistanceFromHome,JobSatisfaction,OverTime,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked
0,2,41,1,4,Yes,0,1,6,0,8
1,1,49,8,2,No,1,3,10,1,1
2,2,37,2,3,Yes,0,3,0,0,6
3,4,33,3,3,Yes,0,3,8,3,1
4,1,27,2,2,No,1,3,2,2,9


In [93]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=78)    

In [94]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
from sklearn.preprocessing import LabelEncoder

# Categorical and Numerical Cols
categorical_cols = ['OverTime', 'Department', 'EducationField', 'JobRole', 'BusinessTravel']
numerical_cols = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
                  'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
                  'YearsSinceLastPromotion', 'NumCompaniesWorked']

X_numeric = attrition_df.copy()

# Label Encode
label_encoder = LabelEncoder()
X_numeric['OverTime'] = label_encoder.fit_transform(X_numeric['OverTime'])

# Find Columns
X_numeric = pd.get_dummies(X_numeric, columns=['Department', 'EducationField', 'JobRole', 'BusinessTravel'])

# Ensure X_columns is defined
X_numeric = X_numeric[[c for c in X_numeric.columns if c in x_columns or c.startswith(('Department_', 'EducationField_', 'JobRole_', 'BusinessTravel_'))]]

print(X_numeric.dtypes)
print(X_numeric.head())

X_df = X_numeric

Age                                  int64
DistanceFromHome                     int64
Education                            int64
JobSatisfaction                      int64
NumCompaniesWorked                   int64
OverTime                             int32
StockOptionLevel                     int64
WorkLifeBalance                      int64
YearsAtCompany                       int64
YearsSinceLastPromotion              int64
Department_Human Resources            bool
Department_Research & Development     bool
Department_Sales                      bool
EducationField_Human Resources        bool
EducationField_Life Sciences          bool
EducationField_Marketing              bool
EducationField_Medical                bool
EducationField_Other                  bool
EducationField_Technical Degree       bool
JobRole_Healthcare Representative     bool
JobRole_Human Resources               bool
JobRole_Laboratory Technician         bool
JobRole_Manager                       bool
JobRole_Man

In [95]:
# Create a StandardScaler
X_scaler = StandardScaler()

# Fit the StandardScaler to the training data
x_train_transformed = X_df.loc[x_train.index]
X_scaler.fit(x_train_transformed)


# Scale the training and testing data
x_train_scaled = X_scaler.transform(x_train_transformed)
x_test_scaled = X_scaler.transform(X_df.loc[x_test.index])



In [96]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
enc = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
enc.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded = enc.transform(y_train[['Department']])
y_test_encoded = enc.transform(y_test[['Department']])
y_train_encoded


array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

## Part 2: Create, Compile, and Train the Model

In [103]:
# Find the number of columns in the X training data.
input_features = len(x_train.columns)
input_features

# Create the input layer
inputs = layers.Input(shape=(input_features,))

# Create at least two shared layers
shared_1 = layers.Dense(units=10, activation='relu')(inputs)
shared_2 = layers.Dense(units=5, activation='relu')(shared_1)

In [98]:
# Create a branch for Department
# with a hidden layer and an output layer
department_branch = layers.Input(shape=(X_df.filter(like='Department').shape[1],), name='department_input')

# Create the hidden layer
department_hidden = layers.Dense(units=10, activation='relu')(department_branch)

# Create the output layer
department_output = layers.Dense(units=5, activation='softmax', name='department_output')(department_hidden)

In [99]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_input = layers.Input(shape=(X_df.filter(like='Department').shape[1],), name='attrition_input')
attrition_branch = layers.concatenate([department_branch, attrition_input])
# Create the hidden layer
attrition_hidden = layers.Dense(units=10, activation='relu')(attrition_branch)

# Create the output layer
attrition_output = layers.Dense(units=5, activation='sigmoid', name='attrition_output')(attrition_hidden)

In [100]:
# Create the model
model = Model(inputs=inputs, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics=['accuracy'])

# Summarize the model
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ department_input    │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_input     │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 6)         │          0 │ department_input… │
│ (Concatenate)       │                   │            │ attrition_input[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 10)        │         40 │ department_input… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, 10)        │         70 │ concatenate_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 5)         │         55 │ dense_23[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 5)         │         55 │ dense_24[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 220 (880.00 B)

 Trainable params: 220 (880.00 B)

 Non-trainable params: 0 (0.00 B)

In [102]:
# Encode the 'Attrition' column
y_train['Attrition'] = label_encoder.fit_transform(y_train['Attrition'])

# Train the model
model.fit(x=x_train_scaled,
          y={'department_output': y_train_encoded, 'attrition_output': y_train['Attrition']},
          epochs=100,
          shuffle=True,
          verbose=2)


Epoch 1/100


ValueError: Input 0 of layer "functional_7" is incompatible with the layer: expected shape=(None, 10), found shape=(None, 31)

In [ ]:
# Evaluate the model with the testing data


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.7868 - department_output_accuracy: 0.4877 - loss: 4.0711  


[4.038690090179443, 0.7880434989929199, 0.5]

In [ ]:
# Print the accuracy for both department and attrition


Attrition predictions accuracy: 0.7880434989929199
Department predictions accuracy: 0.5


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 